In [1]:
# Calculations to perform ancestral state reconstruction on a 4-tip tree for a binary character
# with an asymmetric Q matrix.

In [1]:
%matplotlib inline
import ivy
import numpy as np
from scipy.linalg import expm

In [2]:
tree = ivy.tree.read("(((A:1,B:1)E:1,C:2)F:1,D:3)R;")
chars = [1,1,0,0]
Q = np.array([[-0.1,0.1],[0.1,-0.1]])

In [3]:
print tree.ascii()

                               ---------------+ A 
                --------------E+                  
 --------------F+              ---------------+ B 
 :              :                                 
R+              ------------------------------+ C 
 :                                                
 ---------------------------------------------+ D 


In [4]:
print Q

[[-0.1  0.1]
 [ 0.1 -0.1]]


In [5]:
for i,n in enumerate(tree.leaves()):
    print n.label, chars[i]

A 1
B 1
C 0
D 0


In [6]:
### Step one: provide downpass likelihoods for each tip for each state. Likelihood of observed state is 1,
# likelihood of non-observed state is 0

# DLXN: Downpass likelihood of state X for node N
DL0A = 0;DL1A = 1;DL0B = 0;DL1B = 1;DL0C = 1;DL1C = 0;DL0D = 1;DL1D = 0

In [7]:
### Step two: calculate transition probability matrices for each branch by 
### performing matrix exponentiation * branch length

# Step two-a: calculate probability matrices for all branch lengths present

# P_T = Transition probability matrix for branch of length T
P_1 = expm(Q * 1)
P_2 = expm(Q * 2)
P_3 = expm(Q * 3)

print P_1
print P_2
print P_3

[[ 0.90936538  0.09063462]
 [ 0.09063462  0.90936538]]
[[ 0.83516002  0.16483998]
 [ 0.16483998  0.83516002]]
[[ 0.77440582  0.22559418]
 [ 0.22559418  0.77440582]]


In [8]:
# Step two-b: assign each node its corresponding probability transition rates based on its length

#PXYN: Probability of transitioning from state X to state Y along the branch subtending node N

# The 1-length branches
P00F = P00E = P00B = P00A = P_1[0,0]
P01F = P01E = P01B = P01A = P_1[0,1]
P11F = P11E = P11B = P11A = P_1[1,1]
P10F = P10E = P10B = P10A = P_1[1,0]

# The 2-length branch
P00C = P_2[0,0]
P01C = P_2[0,1]
P11C = P_2[1,1]
P10C = P_2[1,0]

# The 3-length branch
P00D = P_3[0,0]
P01D = P_3[0,1]
P11D = P_3[1,1]
P10D = P_3[1,0]

In [9]:
# Step three: Calculate the down-pass (conditional) likelihood of each internal node

# First we calculate "partial" likelihoods of each node being in a given state using only information
# from one child. 

# Then we calculate the full conditional likelihood by taking the product of these partial likelihoods

# Node E
# P_DLXN_M: "Partial" down-pass likelihood of node N being in state X given only information from child node M
P_DL0E_A = (P00A * DL0A + P01A * DL1A)
P_DL0E_B = (P00B * DL0B + P01B * DL1B)

P_DL1E_A = (P10A * DL0A + P11A * DL1A)
P_DL1E_B = (P10B * DL0B + P11B * DL1B)

# DLXN: Full down-pass likelihood of node N being in state X
DL0E = P_DL0E_A * P_DL0E_B
DL1E = P_DL1E_A * P_DL1E_B

# Node F
P_DL0F_E = (P00E * DL0E + P01E * DL1E)
P_DL0F_C = (P00C * DL0C + P01C * DL1C)

P_DL1F_E = (P10E * DL0E + P11E * DL1E)
P_DL1F_C = (P10C * DL0C + P11C * DL1C)

DL0F = P_DL0F_E * P_DL0F_C
DL1F = P_DL1F_E * P_DL1F_C

# Node R (root)
P_DL0R_F = (P00F * DL0F + P01F * DL1F)
P_DL0R_D = (P00D * DL0D + P01D * DL1D)

P_DL1R_F = (P10F * DL0F + P11F * DL1F)
P_DL1R_D = (P10D * DL0D + P11D * DL1D)

DL0R = P_DL0R_F * P_DL0R_D
DL1R = P_DL1R_F * P_DL1R_D

print DL0E, DL1E
print DL0F, DL1F
print DL0R, DL1R

0.00821463496992 0.826945388048
0.0688338794853 0.124081649965
0.0571830861465 0.026862466839


In [20]:
# Step four: Calculate up-pass likelihood for each node


# Up-pass likelihood has two parts:
# First part: Up-pass-partial-downpass
# This is equivalent to the partial downpass likelihood of the node's parent given only the node's siblings.

# Second part: Up-pass-partial-uppass
# This is equivalent to the partial upass likelihood of the node's parent given only the node's
# parent's siblings. There is no uppass likelihood for the root, so all children of the root have this set to 1


# Node F
# ULXM_m_N_PD = up-pass-partial-downpass of state X for node N's parent M not counting information from N (M_m_N)
UL0R_m_F_PD = P_DL0R_D # Note: more children would be used in this calculation if R had more than 2 children.
UL1R_m_F_PD = P_DL1R_D # In this case, only information from F's sibling D is used.

# ULXM_m_N_PU = up-pass-partial-uppass of state X for node N's parent M not counting information from M.

# We need to calculate the equilibrium frequency of the Q matrix to use as the
# up-pass likelihood at the root
# There is a function in ivy that does this
R_EQ = ivy.chars.mk.qsd(Q)
# R_EQX = equilibrium frequency of state X
R_EQ0 = R_EQ[0]
R_EQ1 = R_EQ[1]

UL0R_m_F_PU = R_EQ0 # Set to EQUILIBRIUM FREQUENCY because F's parent is the root
UL1R_m_F_PU = R_EQ1

# ULXM_m_N = Up-pass likelihood of node N's parent M for state X
UL0R_m_F = UL0R_m_F_PD * UL0R_m_F_PU
UL1R_m_F = UL1R_m_F_PD * UL1R_m_F_PU

# ULXN = Up-pass likelihood of node N being in state X
UL0F = UL0R_m_F * P00F + UL1R_m_F * P10F
UL1F = UL0R_m_F * P01F + UL1R_m_F * P11F

# Node E
UL0F_m_E_PD = P_DL0F_C
UL1F_m_E_PD = P_DL1F_C

# When parent is not the root, up-pass-partial-uppass likelihood is equivalent to the up-pass
# likelihood of the parent node (information from parent node - parent node) times the transition
# probability of the parent's state to the child's state
UL0F_m_E_PU = (UL0R_m_F*P00F) + (UL1R_m_F * P10F)
UL1F_m_E_PU = (UL0R_m_F*P01F) + (UL1R_m_F * P11F)

UL0F_m_E =  UL0F_m_E_PD * UL0F_m_E_PU
UL1F_m_E =  UL1F_m_E_PD * UL1F_m_E_PU

UL0E = UL0F_m_E * P00E + UL1F_m_E * P10E
UL1E = UL0F_m_E * P01E + UL1F_m_E * P11E

In [21]:
print UL0F, UL1F
print UL0R_m_F, UL1R_m_F
print UL0E, UL1E
print UL0F_m_E, UL1F_m_E


0.362332241029 0.137667758971
0.387202909024 0.112797090976
0.277235661147 0.0480628918308
0.302605402758 0.0226931502199


In [23]:
# Step five: calculate the marginal at each root
# Marginal is equivalent to up-pass * down-pass

# MLXN = marginal likelihood of node N being in state X
ML0F = UL0F * DL0F
ML1F = UL1F * DL1F

ML0E = UL0E * DL0E
ML1E = UL1E * DL1E

print ML0F, ML1F
print ML0E, ML1E
print
print ML0F/(ML0F + ML1F), ML1F/(ML0F + ML1F)
print ML0E/(ML0E + ML1E), ML1E/(ML0E + ML1E)

0.0249407338127 0.0170820426801
0.00227738975697 0.0397453867358

0.593505139219 0.406494860781
0.0541941762787 0.945805823721
